# Unsloth: Continued Pretraining (New Language)

This notebook demonstrates how to perform **Continued Pretraining** to teach an LLM a new language (e.g., Hindi) or a new domain.

Unlike instruction tuning, here we train on raw text data to improve the model's fundamental understanding of the new language.

In [1]:
%%capture
!pip install unsloth
!pip install --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-bnb-4bit", # Using a 1B model for speed
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

### LoRA Configuration
We target all linear layers to maximize the model's ability to adapt to the new language structure.

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Higher rank is better for learning new languages
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "embed_tokens", "lm_head"], # Train embeddings and head too!
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:1222: UserWarning: Model has `tie_word_embeddings=True` and a tied layer is part of the adapter, but `ensure_weight_tying` is not set to True. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. Check the discussion here: https://github.com/huggingface/peft/issues/2777
  warnings.warn(msg)
Unsloth 2025.11.4 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


### Data Preparation
We will use a subset of the **Oscar** dataset for Hindi (or any target language). We format it as raw text.

In [5]:
from datasets import load_dataset

dataset = load_dataset("gsm8k", "main", split="train")
dataset = dataset.rename_column("question", "prompt")  # <--- ADD THIS LINE

# Simple reward function
def reward_func(prompts, completions, **kwargs):
    rewards = []
    for completion, ground_truth in zip(completions, kwargs["answer"]):
        if ground_truth.strip() in completion:
            rewards.append(1.0)
        else:
            rewards.append(0.0)
    return rewards

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 1. Define the missing formatting function
def formatting_prompts_func(examples):
    output_texts = []
    # Combine prompt and answer into one block of text for pretraining
    for prompt, answer in zip(examples["prompt"], examples["answer"]):
        text = f"{prompt}\n{answer}"
        output_texts.append(text)
    return output_texts

# 2. Pass it to the SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    formatting_func = formatting_prompts_func,  # <--- ADD THIS LINE
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/7473 [00:00<?, ? examples/s]

In [8]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 615,514,112 of 2,113,996,800 (29.12% trained)


Step,Training Loss
1,1.708000
2,1.805300
3,1.750600
4,1.927000
5,1.758000
6,1.504700
7,1.541000
8,1.465200
9,1.530200
10,1.630300


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=60, training_loss=1.4056796113650003, metrics={'train_runtime': 89.4892, 'train_samples_per_second': 5.364, 'train_steps_per_second': 0.67, 'total_flos': 869600833413120.0, 'train_loss': 1.4056796113650003, 'epoch': 0.06422263848006422})